# Apresentação:

    O objetivo aqui é explorar as capacidades da Biblioteca
    Llamaindex na aplicação de RAG e ver o que conseguimos
    extrair dessa biblioteca.

# Configurando a LLM:

In [3]:
# Importações:
import os
import re
import json
import requests
from IPython.display import Audio, display

# Dependendicas para o LLM:
#from langchain import OpenAI
from openai import OpenAI as oai

# Dependendicas para o RAG:
from llama_index.core import Settings
from llama_index.llms.openai import OpenAI
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core.response.pprint_utils import pprint_response
from llama_index.core.tools import QueryEngineTool, ToolMetadata
from llama_index.core.query_engine import SubQuestionQueryEngine
from llama_index.core import (GPTVectorStoreIndex,PromptHelper,
                              ServiceContext,SimpleDirectoryReader)

# Embaddings:
#from langchain.embeddings import HuggingFaceEmbeddings
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.embeddings.langchain import LangchainEmbedding

In [4]:
API_KEY = input("Insira a API_KEY")
os.environ["OPENAI_API_KEY"] = API_KEY

Insira a API_KEY


In [ ]:
# definição do dicionário de configurações:
headers = {'Authorization': f'Bearer {API_KEY}','Content-Type':'application/json'}

# Definindo o recebimento de dados:
link = 'https://api.openai.com/v1/models'
requisicao = requests.get(link,headers=headers) #get para pegar informação, post para enviar uma informação

print(requisicao)

<Response [200]>


In [ ]:
# Extraindo respostas da requisição para JSON:
dic = requisicao.json()
lista_modelos = [var['id'] for var in dic['data']]

# Modelos:
print(f'São {len(lista_modelos)} disponiveis até o momento\n')
print('modelos:\n')
display(lista_modelos)

São 48 disponiveis até o momento

modelos:



['gpt-4o-realtime-preview',
 'gpt-4o-realtime-preview-2024-10-01',
 'o1-mini-2024-09-12',
 'dall-e-2',
 'gpt-4-turbo',
 'gpt-4-1106-preview',
 'gpt-3.5-turbo',
 'gpt-3.5-turbo-0125',
 'gpt-3.5-turbo-instruct',
 'babbage-002',
 'davinci-002',
 'whisper-1',
 'dall-e-3',
 'text-embedding-3-small',
 'gpt-3.5-turbo-16k',
 'gpt-4-0125-preview',
 'gpt-4-turbo-preview',
 'omni-moderation-latest',
 'gpt-4o-2024-05-13',
 'omni-moderation-2024-09-26',
 'tts-1-hd-1106',
 'chatgpt-4o-latest',
 'gpt-4',
 'gpt-4-0613',
 'o1-mini',
 'gpt-4o-mini',
 'gpt-4o-mini-2024-07-18',
 'o1-preview',
 'o1-preview-2024-09-12',
 'tts-1-hd',
 'text-embedding-ada-002',
 'gpt-3.5-turbo-1106',
 'gpt-4o-audio-preview',
 'gpt-4o-audio-preview-2024-10-01',
 'tts-1',
 'tts-1-1106',
 'gpt-3.5-turbo-instruct-0914',
 'text-embedding-3-large',
 'gpt-4o-realtime-preview-2024-12-17',
 'gpt-4o-mini-realtime-preview',
 'gpt-4o-mini-realtime-preview-2024-12-17',
 'gpt-4o-2024-08-06',
 'gpt-4o',
 'gpt-4o-2024-11-20',
 'gpt-4o-audio-

In [ ]:
# Instanciação modelo da OpenAI:
Settings.llm = OpenAI(openai_api_key=API_KEY, model="gpt-4o-mini", temperature=0, max_tokens=256)

# Instanciando o Embbeding da OpenAI:
Settings.embed_model = OpenAIEmbedding(embed_batch_size=42)

# Instanciando o divisor de Sentença da Biblioteca LlamaIndex:
Settings.text_splitter = SentenceSplitter(chunk_size=1024, chunk_overlap=20)

# Auxiliar de Prompt - Gerencia a Janela de Contexto do Modelo:
Settings.prompt_helper = PromptHelper(
    context_window=4096,
    num_output=256,
    chunk_overlap_ratio=0.1,
    chunk_size_limit=None
)

# Preparando o RAG:

**Carregando os dados**

In [ ]:
# MRPT - fast nearest neighbor search with random projection
!pip install git+http://github.com/vioshyvo/mrpt/

  Cloning http://github.com/vioshyvo/mrpt/ to /tmp/pip-req-build-y8j96m_o
  Running command git clone --filter=blob:none --quiet http://github.com/vioshyvo/mrpt/ /tmp/pip-req-build-y8j96m_o
  Resolved http://github.com/vioshyvo/mrpt/ to commit 88cc6f40782ca0f8de7491279766ded01d767861
  Preparing metadata (setup.py) ... done
  Created wheel for mrpt: filename=mrpt-1.0-cp310-cp310-linux_x86_64.whl size=1561925 sha256=24e4be751aa3543c269a8af61a09f18d149d0a889f9dfb583554904cf3bd47e7
  Stored in directory: /tmp/pip-ephem-wheel-cache-0tqlixzu/wheels/61/f0/46/8fd08e2aa4be121079dc3ef4634352680489c4028bdb57e4de
Successfully built mrpt


In [ ]:
# Importando o Texto bruto:
bruto_text = requests.get('https://raw.githubusercontent.com/abjur/constituicao/main/CONSTITUICAO.md').text

# Otimização do Texto:
padrao_capitulo = r'^##\s+(.*)$' # A ideia é pegar tudo que vem depois do ##
sections = re.split(padrao_capitulo, bruto_text, flags=re.MULTILINE)
sections = [section.strip() for section in sections[1:]]

# Transformando a informação numa string:
string = '\n'.join(sections)

# Construindo o txt:
with open('const.txt', 'w') as arq:
    arq.write(string)

**Construindo os Indices**

In [ ]:
documents = SimpleDirectoryReader(input_files=['const.txt']).load_data()
index = GPTVectorStoreIndex.from_documents(documents)
print(f'arquivo com {len(documents)} paginas')
print(f'Documnto com {len(index.docstore.docs)} nós')

arquivo com 1 paginas
Documnto com 160 nós


**Construindo a Engine de Busca**

In [ ]:
# Definido o modetor de Buscas com 3 vizinhos mais proximos:
expert_engine = index.as_query_engine(similarity_top_k=3)

In [ ]:
# Configurando o Motor de Busca:
query_engine_tools = [
    QueryEngineTool(
        query_engine=expert_engine,
        metadata=ToolMetadata(name='Constituição Federal', description="Providencia Conhecimentos sobre o Direito Brasileiro"))]

In [ ]:
# Instanciando as sub querys:
s_engine = SubQuestionQueryEngine.from_defaults(query_engine_tools=query_engine_tools)

# Testando o RAG:

In [ ]:
import nest_asyncio
nest_asyncio.apply()

In [ ]:
# Prompt para o computador:
def prompt(voice = True, voz = "onyx", model="tts-1", autoplay=True):
  '''
    Função construida para deixar
  o processo de resposta com mais
  funcionalidades e mais intuitívo.
  '''
  # Pergunta:
  pergunta = input('Insira o prompt:\n')

  # Pergunta:
  busca = s_engine.query(pergunta)


  if voice == False:
    return busca.response


  response = oai().audio.speech.create(
      model=model,
      voice=voz,
      input=busca.response,
  )

  # Salvando o arquivo de audio:
  response.stream_to_file("output.wav")

  # Reproduzir o áudio:
  display(Audio("output.wav", autoplay=autoplay))
  return busca.response

# Prompt para o celular:
def celPrompt(pergunta, voice = True, voz = "onyx", model="tts-1", autoplay=True):
  '''
    Função construida para deixar
  o processo de resposta com mais
  funcionalidades e mais intuitívo.

  Nota:
    Atualização para funcionar pelo
  celular.
  '''

  # Pergunta:
  busca = s_engine.query(pergunta)


  if voice == False:
    return busca.response


  response = oai().audio.speech.create(
      model=model,
      voice=voz,
      input=busca.response,
  )

  # Salvando o arquivo de audio:
  response.stream_to_file("output.wav")

  # Reproduzir o áudio:
  display(Audio("output.wav", autoplay=autoplay))
  return busca.response

Nota:

    Caso esteja utilizando o código pelo celular, a função input
    do python pode não funcionar (No meu celular não funciona),
    então, é recomendável que se utilize a função celPrompt para
    chamar conversar com a IA e que inserindo como entrada, uma
    string com a pergunta.

In [ ]:
prompt()

Insira o prompt:
Explique os principais mecanismos de controle de constitucionalidade no ordenamento jurídico brasileiro e discuta como eles garantem a supremacia da Constituição Federal.
Generated 5 sub questions.
[Constituição Federal] Q: Quais são os principais mecanismos de controle de constitucionalidade previstos na Constituição Federal do Brasil?
[Constituição Federal] Q: Como o controle de constitucionalidade é exercido no Brasil?
[Constituição Federal] Q: Quais são os efeitos das decisões de controle de constitucionalidade sobre normas infraconstitucionais?
[Constituição Federal] Q: De que forma o controle de constitucionalidade contribui para a proteção dos direitos fundamentais no Brasil?
[Constituição Federal] Q: Como a supremacia da Constituição Federal é assegurada através dos mecanismos de controle de constitucionalidade?
[Constituição Federal] A: As decisões de controle de constitucionalidade, especialmente nas ações diretas de inconstitucionalidade e nas ações declarat

<ipython-input-16-1b06d2a73b32>:26: DeprecationWarning: Due to a bug, this method doesn't actually stream the response content, `.with_streaming_response.method()` should be used instead
  response.stream_to_file("output.wav")


'Os principais mecanismos de controle de constitucionalidade no ordenamento jurídico brasileiro incluem a ação direta de inconstitucionalidade (ADI) e a ação declaratória de constitucionalidade (ADC). Esses instrumentos podem ser propostos por diversas entidades, como o Presidente da República, as Mesas do Senado e da Câmara dos Deputados, governadores, o Procurador-Geral da República, e partidos políticos com representação no Congresso Nacional.\n\nEsses mecanismos garantem a supremacia da Constituição Federal ao permitir que normas infraconstitucionais sejam avaliadas quanto à sua conformidade com os preceitos constitucionais. O Supremo Tribunal Federal (STF) é o responsável por apreciar essas ações, podendo declarar a inconstitucionalidade de normas ou atos normativos. As decisões do STF têm efeito vinculante, o que significa que são obrigatórias para todos os órgãos do Poder Judiciário e para a administração pública em todas as esferas, assegurando uma interpretação uniforme da Con